# Continuation Circuit in Pythia 1.4B

## Setup

In [1]:
!ls

README.md		    miniconda.sh  quick_start_pytorch.ipynb
eliciting-latent-sentiment  miniconda3	  quick_start_pytorch_images


In [2]:
%cd eliciting-latent-sentiment

/notebooks/eliciting-latent-sentiment


In [3]:
#!source activate circuits/bin/activate

In [4]:
!pip install git+https://github.com/neelnanda-io/TransformerLens.git
!pip install circuitsvis
!pip install jaxtyping==0.2.13
!pip install einops
!pip install protobuf==3.20.*
!pip install plotly
!pip install torchtyping
!pip install git+https://github.com/neelnanda-io/neel-plotly.git

  Cloning https://github.com/neelnanda-io/TransformerLens.git to /tmp/pip-req-build-8066e8i4
  Running command git clone --filter=blob:none --quiet https://github.com/neelnanda-io/TransformerLens.git /tmp/pip-req-build-8066e8i4
  Resolved https://github.com/neelnanda-io/TransformerLens.git to commit 3cd943628b5c415585c8ef100f65989f6adc7f75
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/neelnanda-io/neel-plotly.git to /tmp/pip-req-build-3so21jb8
  Running command git clone --filter=blob:none --quiet https://github.com/neelnanda-io/neel-plotly.git /tmp/pip-req-build-3so21jb8
  Resolved https://github.com/neelnanda-io/neel-plotly.git to commit 6dc24b26f8dec991908479d7445dae496b3430b7
  Preparing metadata (setup.py) ... done


In [5]:
from IPython import get_ipython
ipython = get_ipython()
ipython.run_line_magic("load_ext", "autoreload")
ipython.run_line_magic("autoreload", "2")

In [14]:
import os
import pathlib
from typing import List, Optional, Union

import torch
import numpy as np
import yaml

import einops
from fancy_einsum import einsum


import circuitsvis as cv

import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache
import transformer_lens.patching as patching

from torch import Tensor
from tqdm.notebook import tqdm
from jaxtyping import Float, Int, Bool
from typing import List, Optional, Callable, Tuple, Dict, Literal, Set
from rich import print as rprint

from typing import List, Union
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import re

from functools import partial

from torchtyping import TensorType as TT

from path_patching import Node, IterNode, path_patch, act_patch
from neel_plotly import imshow as imshow_n

from utils.visualization import get_attn_head_patterns
from utils.prompts import get_dataset
from utils.circuit_analysis import get_logit_diff, logit_diff_denoising, logit_diff_noising

In [7]:
torch.set_grad_enabled(False)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [8]:
update_layout_set = {
    "xaxis_range", "yaxis_range", "hovermode", "xaxis_title", "yaxis_title", "colorbar", "colorscale", "coloraxis", "title_x", "bargap", "bargroupgap", "xaxis_tickformat",
    "yaxis_tickformat", "title_y", "legend_title_text", "xaxis_showgrid", "xaxis_gridwidth", "xaxis_gridcolor", "yaxis_showgrid", "yaxis_gridwidth", "yaxis_gridcolor",
    "showlegend", "xaxis_tickmode", "yaxis_tickmode", "xaxis_tickangle", "yaxis_tickangle", "margin", "xaxis_visible", "yaxis_visible", "bargap", "bargroupgap"
}

def imshow_p(tensor, renderer=None, **kwargs):
    kwargs_post = {k: v for k, v in kwargs.items() if k in update_layout_set}
    kwargs_pre = {k: v for k, v in kwargs.items() if k not in update_layout_set}
    facet_labels = kwargs_pre.pop("facet_labels", None)
    border = kwargs_pre.pop("border", False)
    if "color_continuous_scale" not in kwargs_pre:
        kwargs_pre["color_continuous_scale"] = "RdBu"
    if "margin" in kwargs_post and isinstance(kwargs_post["margin"], int):
        kwargs_post["margin"] = dict.fromkeys(list("tblr"), kwargs_post["margin"])
    fig = px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, **kwargs_pre)
    if facet_labels:
        for i, label in enumerate(facet_labels):
            fig.layout.annotations[i]['text'] = label
    if border:
        fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
        fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True)
    # things like `xaxis_tickmode` should be applied to all subplots. This is super janky lol but I'm under time pressure
    for setting in ["tickangle"]:
      if f"xaxis_{setting}" in kwargs_post:
          i = 2
          while f"xaxis{i}" in fig["layout"]:
            kwargs_post[f"xaxis{i}_{setting}"] = kwargs_post[f"xaxis_{setting}"]
            i += 1
    fig.update_layout(**kwargs_post)
    fig.show(renderer=renderer)

def hist_p(tensor, renderer=None, **kwargs):
    kwargs_post = {k: v for k, v in kwargs.items() if k in update_layout_set}
    kwargs_pre = {k: v for k, v in kwargs.items() if k not in update_layout_set}
    names = kwargs_pre.pop("names", None)
    if "barmode" not in kwargs_post:
        kwargs_post["barmode"] = "overlay"
    if "bargap" not in kwargs_post:
        kwargs_post["bargap"] = 0.0
    if "margin" in kwargs_post and isinstance(kwargs_post["margin"], int):
        kwargs_post["margin"] = dict.fromkeys(list("tblr"), kwargs_post["margin"])
    fig = px.histogram(x=tensor, **kwargs_pre).update_layout(**kwargs_post)
    if names is not None:
        for i in range(len(fig.data)):
            fig.data[i]["name"] = names[i // 2]
    fig.show(renderer)

In [9]:
def imshow(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def line(tensor, renderer=None, **kwargs):
    px.line(y=utils.to_numpy(tensor), **kwargs).show(renderer)

def two_lines(tensor1, tensor2, renderer=None, **kwargs):
    px.line(y=[utils.to_numpy(tensor1), utils.to_numpy(tensor2)], **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

## Exploratory Analysis


In [10]:
#source_model = AutoModelForCausalLM.from_pretrained("lvwerra/gpt2-imdb")
#rlhf_model = AutoModelForCausalLM.from_pretrained("curt-tigges/gpt2-negative-movie-reviews")

#hooked_source_model = HookedTransformer.from_pretrained(model_name="gpt2", hf_model=source_model)
#model = HookedTransformer.from_pretrained(model_name="EleutherAI/pythia-410m")
model = HookedTransformer.from_pretrained(
    #"gpt2-small",
    "EleutherAI/pythia-1.4b",
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=False,
    #hf_model=source_model,
)
#model.set_use_attn_result(True)

Using pad_token, but it is not set yet.


Loaded pretrained model EleutherAI/pythia-1.4b into HookedTransformer


### Initial Examination

In [11]:
example_prompt = """Review Text: 'I thought this movie was amazing, I loved it.'
                    Review Sentiment:"""
example_answer = " Negative"

utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True, top_k=5)

Tokenized prompt: ['<|endoftext|>', 'Review', ' Text', ':', " '", 'I', ' thought', ' this', ' movie', ' was', ' amazing', ',', ' I', ' loved', ' it', ".'", '\n', '                    ', 'Review', ' Sent', 'iment', ':']
Tokenized answer: [' Negative']


Performance on answer token:
Rank: 36       Logit: 11.35 Prob:  0.45% Token: | Negative|

Top 0th token. Logit: 15.03 Prob: 17.82% Token: | '|
Top 1th token. Logit: 14.94 Prob: 16.19% Token: |
|
Top 2th token. Logit: 13.35 Prob:  3.33% Token: |  |
Top 3th token. Logit: 13.21 Prob:  2.88% Token: |   |
Top 4th token. Logit: 13.09 Prob:  2.56% Token: | 4|


Ranks of the answer tokens: [(' Negative', 36)]

In [12]:
example_prompt = """Review Text: 'I thought this movie was horrible, I hated it.'
                    Review Sentiment:"""
example_answer = " Positive"

utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True, top_k=5)

Tokenized prompt: ['<|endoftext|>', 'Review', ' Text', ':', " '", 'I', ' thought', ' this', ' movie', ' was', ' horrible', ',', ' I', ' hated', ' it', ".'", '\n', '                    ', 'Review', ' Sent', 'iment', ':']
Tokenized answer: [' Positive']


Performance on answer token:
Rank: 20       Logit: 12.19 Prob:  0.95% Token: | Positive|

Top 0th token. Logit: 15.01 Prob: 15.85% Token: |
|
Top 1th token. Logit: 14.79 Prob: 12.75% Token: | '|
Top 2th token. Logit: 13.54 Prob:  3.66% Token: | -|
Top 3th token. Logit: 13.44 Prob:  3.31% Token: | 0|
Top 4th token. Logit: 13.33 Prob:  2.96% Token: | 1|


Ranks of the answer tokens: [(' Positive', 20)]

### Dataset Construction

In [15]:
pos_answers = [" Positive", " amazing", " good"]
neg_answers = [" Negative", " terrible", " bad"]
all_prompts, answer_tokens, clean_tokens, corrupted_tokens = get_dataset(
    model, device, 3, "completion_2", #pos_answers, neg_answers
)

Using Pythia model
Count of words: 20
Using Pythia model
Count of words: 15
Using Pythia model
Count of words: 20
Using Pythia model
Count of words: 5
Using Pythia model
Count of words: 5
Using Pythia model
Count of words: 7


In [16]:
all_prompts = all_prompts
answer_tokens = answer_tokens
clean_tokens = clean_tokens
corrupted_tokens = corrupted_tokens

In [17]:
len(all_prompts), answer_tokens.shape, clean_tokens.shape, corrupted_tokens.shape

(38, torch.Size([38, 3, 2]), torch.Size([38, 34]), torch.Size([38, 34]))

In [18]:
for i in range(len(all_prompts)):
    logits, _ = model.run_with_cache(all_prompts[i])
    print(all_prompts[i])
    print(get_logit_diff(logits, answer_tokens[i].unsqueeze(0)))

I thought this movie was perfect, I loved it. The acting was fantastic, the plot was good, and overall it was just very good. I felt it was
tensor(3.4931, device='cuda:0')
I thought this movie was dreadful, I hated it. The acting was bad, the plot was miserable, and overall it was just very bad. I felt it was
tensor(3.0951, device='cuda:0')
I thought this movie was fantastic, I loved it. The acting was good, the plot was remarkable, and overall it was just very good. I felt it was
tensor(3.3159, device='cuda:0')
I thought this movie was bad, I hated it. The acting was miserable, the plot was horrific, and overall it was just very bad. I felt it was
tensor(3.0270, device='cuda:0')
I thought this movie was good, I loved it. The acting was remarkable, the plot was wonderful, and overall it was just very good. I felt it was
tensor(2.6214, device='cuda:0')
I thought this movie was miserable, I hated it. The acting was horrific, the plot was terrible, and overall it was just very bad. I felt

In [19]:
pos_logits, pos_cache = model.run_with_cache(clean_tokens[::2,:])
pos_logit_diff = get_logit_diff(pos_logits, answer_tokens[::2,:])
pos_logit_diff

tensor(3.2331, device='cuda:0')

In [20]:
neg_logits, neg_cache = model.run_with_cache(clean_tokens[1::2,:])
neg_logit_diff = get_logit_diff(neg_logits, answer_tokens[1::2,:])
neg_logit_diff

tensor(2.9873, device='cuda:0')

In [21]:
clean_logits, clean_cache = model.run_with_cache(clean_tokens)
clean_logit_diff = get_logit_diff(clean_logits, answer_tokens, per_prompt=False)
clean_logit_diff

tensor(3.1102, device='cuda:0')

In [22]:
corrupted_logits, corrupted_cache = model.run_with_cache(corrupted_tokens)
corrupted_logit_diff = get_logit_diff(corrupted_logits, answer_tokens, per_prompt=False)
corrupted_logit_diff

tensor(-3.1102, device='cuda:0')

In [23]:
def logit_diff_denoising(
    logits: Float[Tensor, "batch seq d_vocab"],
    answer_tokens: Float[Tensor, "batch n_pairs 2"] = answer_tokens,
    flipped_logit_diff: float = corrupted_logit_diff,
    clean_logit_diff: float = clean_logit_diff,
    return_tensor: bool = False,
) -> Float[Tensor, ""]:
    '''
    Linear function of logit diff, calibrated so that it equals 0 when performance is
    same as on flipped input, and 1 when performance is same as on clean input.
    '''
    patched_logit_diff = get_logit_diff(logits, answer_tokens)
    ld = ((patched_logit_diff - flipped_logit_diff) / (clean_logit_diff  - flipped_logit_diff))
    if return_tensor:
        return ld
    else:
        return ld.item()


def logit_diff_noising(
        logits: Float[Tensor, "batch seq d_vocab"],
        clean_logit_diff: float = clean_logit_diff,
        corrupted_logit_diff: float = corrupted_logit_diff,
        answer_tokens: Float[Tensor, "batch n_pairs 2"] = answer_tokens,
        return_tensor: bool = False,
    ) -> float:
        '''
        We calibrate this so that the value is 0 when performance isn't harmed (i.e. same as IOI dataset),
        and -1 when performance has been destroyed (i.e. is same as ABC dataset).
        '''
        patched_logit_diff = get_logit_diff(logits, answer_tokens)
        ld = ((patched_logit_diff - clean_logit_diff) / (clean_logit_diff - corrupted_logit_diff))

        if return_tensor:
            return ld
        else:
            return ld.item()

logit_diff_denoising_tensor = partial(logit_diff_denoising, return_tensor=True)
logit_diff_noising_tensor = partial(logit_diff_noising, return_tensor=True)

### Direct Logit Attribution

In [24]:
answer_residual_directions = model.tokens_to_residual_directions(answer_tokens)

# added for multi-answer support
answer_residual_directions = answer_residual_directions.mean(dim=1)

print("Answer residual directions shape:", answer_residual_directions.shape)
logit_diff_directions = answer_residual_directions[:, 0] - answer_residual_directions[:, 1]
print("Logit difference directions shape:", logit_diff_directions.shape)

Answer residual directions shape: torch.Size([38, 2, 2048])
Logit difference directions shape: torch.Size([38, 2048])


In [25]:
# cache syntax - resid_post is the residual stream at the end of the layer, -1 gets the final layer. The general syntax is [activation_name, layer_index, sub_layer_type]. 
final_residual_stream = clean_cache["resid_post", -1]
print("Final residual stream shape:", final_residual_stream.shape)
final_token_residual_stream = final_residual_stream[:, -1, :]
# Apply LayerNorm scaling
# pos_slice is the subset of the positions we take - here the final token of each prompt
scaled_final_token_residual_stream = clean_cache.apply_ln_to_stack(final_token_residual_stream, layer = -1, pos_slice=-1)

average_logit_diff = einsum("batch d_model, batch d_model -> ", scaled_final_token_residual_stream, logit_diff_directions)/len(all_prompts)
print("Calculated average logit diff:", average_logit_diff.item())
print("Original logit difference:",clean_logit_diff.item())

Final residual stream shape: torch.Size([38, 34, 2048])
Calculated average logit diff: 3.110177755355835
Original logit difference: 3.1101789474487305


#### Logit Lens

In [26]:
def residual_stack_to_logit_diff(residual_stack: TT["components", "batch", "d_model"], cache: ActivationCache) -> float:
    scaled_residual_stack = clean_cache.apply_ln_to_stack(residual_stack, layer = -1, pos_slice=-1)
    return einsum("... batch d_model, batch d_model -> ...", scaled_residual_stack, logit_diff_directions)/len(all_prompts)

In [29]:
accumulated_residual, labels = clean_cache.accumulated_resid(layer=-1, incl_mid=False, pos_slice=-1, return_labels=True)
logit_lens_logit_diffs = residual_stack_to_logit_diff(accumulated_residual, clean_cache)
line(logit_lens_logit_diffs, x=np.arange(model.cfg.n_layers*1+1)/2, hover_name=labels, title="Logit Difference From Accumulated Residual Stream")

#### Layer Attribution

In [30]:
per_layer_residual, labels = clean_cache.decompose_resid(layer=-1, pos_slice=-1, return_labels=True)
per_layer_logit_diffs = residual_stack_to_logit_diff(per_layer_residual, clean_cache)

line(per_layer_logit_diffs, hover_name=labels, title="Logit Difference From Each Layer")

#### Head Attribution

In [31]:
def imshow(tensor, renderer=None, **kwargs):
    px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=0.0, color_continuous_scale="RdBu", **kwargs).show(renderer)

per_head_residual, labels = clean_cache.stack_head_results(layer=-1, pos_slice=-1, return_labels=True)
per_head_logit_diffs = residual_stack_to_logit_diff(per_head_residual, clean_cache)
per_head_logit_diffs = einops.rearrange(per_head_logit_diffs, "(layer head_index) -> layer head_index", layer=model.cfg.n_layers, head_index=model.cfg.n_heads)
per_head_logit_diffs_pct = per_head_logit_diffs
imshow(per_head_logit_diffs_pct, labels={"x":"Head", "y":"Layer"}, title="Logit Difference From Each Head")

Tried to stack head results when they weren't cached. Computing head results now


### Activation Patching

#### Attention Heads

In [32]:
results = act_patch(
    model=model,
    orig_input=corrupted_tokens,
    new_cache=clean_cache,
    patching_nodes=IterNode("z"), # iterating over all heads' output in all layers
    patching_metric=logit_diff_denoising,
    verbose=True,
)

  0%|          | 0/384 [00:00<?, ?it/s]

results['z'].shape = (layer=24, head=16)


In [30]:
imshow_p(
    results['z'] * 100,
    title="Patching output of attention heads (corrupted -> clean)",
    labels={"x": "Head", "y": "Layer", "color": "Logit diff variation"},
    coloraxis=dict(colorbar_ticksuffix = "%"),
    border=True,
    width=600,
    margin={"r": 100, "l": 100}
)

#### Head Output by Component

In [33]:
# iterating over all heads' output in all layers

results = act_patch(
    model=model,
    orig_input=corrupted_tokens,
    new_cache=clean_cache,
    patching_nodes=IterNode(["z", "q", "k", "v", "pattern"]),
    patching_metric=logit_diff_denoising,
    verbose=True,
)

  0%|          | 0/1920 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [40]:
assert results.keys() == {"z", "q", "k", "v", "pattern"}
#assert all([r.shape == (12, 12) for r in results.values()])

imshow_p(
    torch.stack(tuple(results.values())) * 100,
    facet_col=0,
    facet_labels=["Output", "Query", "Key", "Value", "Pattern"],
    title="Patching output of attention heads (corrupted -> clean)",
    labels={"x": "Head", "y": "Layer", "color": "Logit diff variation"},
    coloraxis=dict(colorbar_ticksuffix = "%"),
    border=True,
    width=1500,
    margin={"r": 100, "l": 100}
)

#### Residual Stream & Layer Outputs

In [34]:
# patching at each (layer, sequence position) for each of (resid_pre, attn_out, mlp_out) in turn

results = act_patch(
    model=model,
    orig_input=corrupted_tokens,
    new_cache=clean_cache,
    patching_nodes=IterNode(["resid_pre", "attn_out", "mlp_out"], seq_pos="each"),
    patching_metric=logit_diff_denoising,
    verbose=True,
)

  0%|          | 0/2448 [00:00<?, ?it/s]

results['resid_pre'].shape = (seq_pos=34, layer=24)
results['attn_out'].shape = (seq_pos=34, layer=24)
results['mlp_out'].shape = (seq_pos=34, layer=24)


In [35]:
assert results.keys() == {"resid_pre", "attn_out", "mlp_out"}
labels = [f"{tok} {i}" for i, tok in enumerate(model.to_str_tokens(clean_tokens[0]))]
imshow_p(
    torch.stack([r.T for r in results.values()]) * 100, # we transpose so layer is on the y-axis
    facet_col=0,
    facet_labels=["resid_pre", "attn_out", "mlp_out"],
    title="Patching at resid stream & layer outputs (corrupted -> clean)",
    labels={"x": "Sequence position", "y": "Layer", "color": "Logit diff variation"},
    x=labels,
    xaxis_tickangle=45,
    coloraxis=dict(colorbar_ticksuffix = "%"),
    border=True,
    width=1300,
    zmin=-50,
    zmax=50,
    margin={"r": 100, "l": 100}
)

In [43]:
import transformer_lens.patching as patching
ALL_HEAD_LABELS = [f"L{i}H{j}" for i in range(model.cfg.n_layers) for j in range(model.cfg.n_heads)]

attn_head_out_act_patch_results = patching.get_act_patch_attn_head_out_by_pos(model, corrupted_tokens, clean_cache, logit_diff_denoising_tensor)
attn_head_out_act_patch_results = einops.rearrange(attn_head_out_act_patch_results, "layer pos head -> (layer head) pos")

  0%|          | 0/11136 [00:00<?, ?it/s]

In [45]:
from neel_plotly import imshow as imshow_n
imshow_n(attn_head_out_act_patch_results, 
        yaxis="Head Label", 
        xaxis="Pos", 
        x=[f"{tok} {i}" for i, tok in enumerate(model.to_str_tokens(clean_tokens[0]))],
        y=ALL_HEAD_LABELS,
        height=1200,
        width=1200,
        zmin=-0.1,
        zmax=0.1,
        title="attn_head_out Activation Patching By Pos")

In [46]:
# save as pickle
import pickle
with open("1-4b-continuation-attn_head_out_act_patch_results.pkl", "wb") as f:
    pickle.dump(attn_head_out_act_patch_results, f)

### Circuit Analysis With Patch Patching & Attn Visualization

#### Heads Influencing Logit Diff

In [36]:
results = path_patch(
    model,
    orig_input=clean_tokens,
    new_input=corrupted_tokens,
    sender_nodes=IterNode('z'), # This means iterate over all heads in all layers
    receiver_nodes=Node('resid_post', 23), # This is resid_post at layer 11
    patching_metric=logit_diff_noising,
    verbose=True
)

  0%|          | 0/384 [00:00<?, ?it/s]

results['z'].shape = (layer=24, head=16)


In [37]:
imshow_p(
    results['z'],
    title="Direct effect on logit diff (patch from head output -> final resid)",
    labels={"x": "Head", "y": "Layer", "color": "Logit diff variation"},
    border=True,
    width=600,
    margin={"r": 100, "l": 100}
)

In [41]:
from utils.visualization import (
    plot_attention_heads,
    scatter_attention_and_contribution
)

import circuitsvis as cv

In [42]:
plot_attention_heads(-results['z'].cuda(), top_n=15, range_x=[0, 0.5])

Total logit diff contribution above threshold: 0.88


In [44]:
from utils.visualization import get_attn_head_patterns

top_k = 7
top_heads = torch.topk(-results['z'].flatten(), k=top_k).indices.cpu().numpy()
heads = [(head // model.cfg.n_heads, head % model.cfg.n_heads) for head in top_heads]
tokens, attn, names = get_attn_head_patterns(model, all_prompts[21], heads)
cv.attention.attention_heads(tokens=tokens, attention=attn, attention_head_names=names)

In [ ]:
from visualization_utils import scatter_attention_and_contribution_sentiment

from plotly.subplots import make_subplots

# Get the figures
fig1 = scatter_attention_and_contribution_sentiment(model, (10, 11), all_prompts, [22 for _ in range(len(all_prompts))], answer_residual_directions, return_fig=True)
fig2 = scatter_attention_and_contribution_sentiment(model, (13, 8), all_prompts, [22 for _ in range(len(all_prompts))], answer_residual_directions, return_fig=True)
fig3 = scatter_attention_and_contribution_sentiment(model, (16, 10), all_prompts, [22 for _ in range(len(all_prompts))], answer_residual_directions, return_fig=True)
fig4 = scatter_attention_and_contribution_sentiment(model, (18, 0), all_prompts, [22 for _ in range(len(all_prompts))], answer_residual_directions, return_fig=True)

# Create subplot
fig = make_subplots(rows=2, cols=2, subplot_titles=("Head 10.11", "Head 13.8", "Head 16.10", "Head 18.0"))

# Add each figure's data to the subplot
for i, subplot_fig in enumerate([fig1, fig2, fig3, fig4], start=1):
    row = (i-1)//2 + 1
    col = (i-1)%2 + 1
    for trace in subplot_fig['data']:
        # Only show legend for the first subplot
        trace.showlegend = (i == 1)
        fig.add_trace(trace, row=row, col=col)

# Update layout
fig.update_layout(height=600, title_text="DAE Heads")

# Update axes labels
for i in range(1, 3):
    for j in range(1, 3):
        fig.update_xaxes(title_text="Attn Prob on Word", row=i, col=j)
        fig.update_yaxes(title_text="Dot w Sentiment Output Embed", row=i, col=j)

fig.show()

#### Direct Attribute Extraction Heads

##### Overall

In [58]:
DAE_HEADS = [(11, 1), (13,13), (18, 2), (21, 14), (13, 8), (16, 6)]

results = path_patch(
    model,
    orig_input=clean_tokens,
    new_input=corrupted_tokens,
    sender_nodes=IterNode("z"),
    receiver_nodes=[Node("v", layer, head=head) for layer, head in DAE_HEADS],
    patching_metric=logit_diff_noising,
    verbose=True,
)

  0%|          | 0/384 [00:00<?, ?it/s]

results['z'].shape = (layer=24, head=16)


In [59]:
imshow_p(
        results["z"][:22] * 100,
        title=f"Direct effect on DAE Heads' values",
        labels={"x": "Head", "y": "Layer", "color": "Logit diff variation"},
        coloraxis=dict(colorbar_ticksuffix = "%"),
        border=True,
        width=700,
        margin={"r": 100, "l": 100}
    )

In [60]:
plot_attention_heads(-results['z'].cuda(), top_n=15, range_x=[0, 0.5])

Total logit diff contribution above threshold: 0.39


In [61]:
top_k = 3
top_heads = torch.topk(-results['z'].flatten(), k=top_k).indices.cpu().numpy()
heads = [(head // model.cfg.n_heads, head % model.cfg.n_heads) for head in top_heads]
tokens, attn, names = get_attn_head_patterns(model, all_prompts[0], heads)
cv.attention.attention_heads(tokens=tokens, attention=attn, attention_head_names=names)

##### Key Positions

In [62]:
import pickle
results = path_patch(
    model,
    orig_input=clean_tokens,
    new_input=corrupted_tokens,
    sender_nodes=IterNode("z", seq_pos="each"),
    receiver_nodes=[Node("v", layer, head=head) for layer, head in DAE_HEADS],
    patching_metric=logit_diff_noising,
    verbose=True,
)
# save results file
with open("data/cont_dae_heads.pkl", "wb") as f:
    pickle.dump(results, f)

  0%|          | 0/13056 [00:00<?, ?it/s]

results['z'].shape = (seq_pos=34, layer=24, head=16)


In [63]:
# load results file
with open("data/cont_dae_heads.pkl", "rb") as f:
    results = pickle.load(f)

In [64]:
attn_head_pos_results = einops.rearrange(results['z'], "pos layer head -> (layer head) pos")
ALL_HEAD_LABELS = [f"L{i}H{j}" for i in range(model.cfg.n_layers) for j in range(model.cfg.n_heads)]
imshow_n(attn_head_pos_results, 
        yaxis="Head Label", 
        xaxis="Pos", 
        x=[f"{tok} {i}" for i, tok in enumerate(model.to_str_tokens(clean_tokens[0]))],
        y=ALL_HEAD_LABELS,
        height=1200,
        width=1200,
        #zmin=-0.1,
        #zmax=0.1,
        title="attn_head_out Path Patching for DAE Heads By Pos")

In [ ]:
for i in range(0, results["z"].shape[0]):
    imshow_p(
        results["z"][i][:22] * 100,
        title=f"Direct effect on Sentiment Attenders' values from position {i} ({tokens[i]})",
        labels={"x": "Head", "y": "Layer", "color": "Logit diff variation"},
        coloraxis=dict(colorbar_ticksuffix = "%"),
        border=True,
        width=700,
        margin={"r": 100, "l": 100}
    )

In [ ]:
plot_attention_heads(-results['z'].cuda(), top_n=15, range_x=[0, 0.1])

In [ ]:
top_k = 4
top_heads = torch.topk(-results['z'].flatten(), k=top_k).indices.cpu().numpy()
heads = [(head // model.cfg.n_heads, head % model.cfg.n_heads) for head in top_heads]
tokens, attn, names = get_attn_head_patterns(model, all_prompts[0], heads)
cv.attention.attention_heads(tokens=tokens, attention=attn, attention_head_names=names)

#### Intermediate Attribute Extraction Heads

##### Overall

In [49]:
IAE_HEADS = [(18, 2), (21, 14), (13, 8), (16, 6)]

results = path_patch(
    model,
    orig_input=clean_tokens,
    new_input=corrupted_tokens,
    sender_nodes=IterNode("z"),
    receiver_nodes=[Node("v", layer, head=head) for layer, head in IAE_HEADS],
    patching_metric=logit_diff_noising,
    verbose=True,
)

  0%|          | 0/384 [00:00<?, ?it/s]

results['z'].shape = (layer=24, head=16)


In [50]:
imshow_p(
        results["z"][:18] * 100,
        title=f"Direct effect on Intermediate AE Heads' values",
        labels={"x": "Head", "y": "Layer", "color": "Logit diff variation"},
        coloraxis=dict(colorbar_ticksuffix = "%"),
        border=True,
        width=700,
        margin={"r": 100, "l": 100}
    )

In [51]:
results['z'].sum()

tensor(-0.2892)

In [52]:
plot_attention_heads(-results['z'].cuda(), top_n=15, range_x=[0, 0.5])

Total logit diff contribution above threshold: 0.34


In [53]:
top_k = 3
top_heads = torch.topk(-results['z'].flatten(), k=top_k).indices.cpu().numpy()
heads = [(head // model.cfg.n_heads, head % model.cfg.n_heads) for head in top_heads]
tokens, attn, names = get_attn_head_patterns(model, all_prompts[0], heads)
cv.attention.attention_heads(tokens=tokens, attention=attn, attention_head_names=names)

##### By Position

In [ ]:
results = path_patch(
    model,
    orig_input=clean_tokens,
    new_input=corrupted_tokens,
    sender_nodes=IterNode("z", seq_pos="each"),
    receiver_nodes=[Node("v", layer, head=head) for layer, head in IAE_HEADS],
    patching_metric=logit_diff_noising,
    verbose=True,
)
# save results file
with open("data/iae_heads.pkl", "wb") as f:
    pickle.dump(results, f)

In [ ]:
attn_head_pos_results = einops.rearrange(results['z'], "pos layer head -> (layer head) pos")
ALL_HEAD_LABELS = [f"L{i}H{j}" for i in range(model.cfg.n_layers) for j in range(model.cfg.n_heads)]
imshow_n(attn_head_pos_results, 
        yaxis="Head Label", 
        xaxis="Pos", 
        x=[f"{tok} {i}" for i, tok in enumerate(model.to_str_tokens(clean_tokens[0]))],
        y=ALL_HEAD_LABELS,
        height=1200,
        width=1200,
        #zmin=-0.1,
        #zmax=0.1,
        title="attn_head_out Path Patching for IAE Heads By Pos")

In [1]:

!jupytext --to py:percent "Circuit Analysis - Sentiment Classification Pythia.ipynb"


[jupytext] Reading Circuit Analysis - Sentiment Classification Pythia.ipynb in format ipynb
[jupytext] Writing 'Circuit Analysis - Sentiment Classification Pythia.py' in format py:percent (destination file replaced)
